<a href="https://colab.research.google.com/github/CiaraAOC/frictionless/blob/main/frictionless.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Set-up:**

Install modules for frictionless, api retrieval and excel 

In [1]:
!pip install frictionless 
import frictionless
!pip install pyjstat
import pyjstat
from pyjstat import pyjstat
import pandas as pd
from pprint import pprint

     |████████████████████████████████| 234 kB 5.3 MB/s 
     |████████████████████████████████| 636 kB 46.4 MB/s 
     |████████████████████████████████| 245 kB 47.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 45 kB 3.3 MB/s 
  Created wheel for petl: filename=petl-1.7.4-py3-none-any.whl size=217001 sha256=af5488ff59772eb7978d139b8a40b64be6de2163d7d9d927d5f6cbf4557f66dd
  Stored in directory: /root/.cache/pip/wheels/75/23/fb/a0e8e350a8718579b0cde55163df77b94de835d7d116388fa5
  Created wheel for simpleeval: filename=simpleeval-0.9.10-py3-none-any.whl size=13769 sha256=855674f5f1026bd565630045aa90bce0a4a1082cec6bd3c6018cef490d2e4a72
  Stored in directory: /root/.cache/pip/wheels/d4/aa/50/0b420d1eabad3c16a82368935c6a2050955bc3ee2a11ee4e06
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3587 sha256=d811a60105b8c00f411418fba73688e0e

In [2]:
!pip install openpyxl
from openpyxl import Workbook

In [3]:
wb = Workbook()
ws = wb.active
ws.title = "TFA22"
wb.save(filename = 'TFA22.xlsx')

In [4]:
from pyjstat import pyjstat
dataset = pyjstat.Dataset.read('https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/TFA22/JSON-stat/2.0/en')

In [5]:
df = dataset.write('dataframe')
print(df)

               Statistic  Year  ...           Region of Origin    value
0        Tonne-kilometre  2016  ...  All regions and countries  11564.0
1        Tonne-kilometre  2016  ...                     Border    887.0
2        Tonne-kilometre  2016  ...                       West    839.0
3        Tonne-kilometre  2016  ...                   Mid-West   1088.0
4        Tonne-kilometre  2016  ...                 South-East   1077.0
...                  ...   ...  ...                        ...      ...
1810  Vehicle Kilometres  2020  ...                     Dublin     10.0
1811  Vehicle Kilometres  2020  ...                   Mid-East      4.0
1812  Vehicle Kilometres  2020  ...                    Midland      1.0
1813  Vehicle Kilometres  2020  ...           Northern Ireland      2.0
1814  Vehicle Kilometres  2020  ...            Other countries     55.0

[1815 rows x 5 columns]


**Step 1: Describe**

Generates metadata describing layout/contents of dataset

In [6]:
from frictionless import describe 

resource = describe(df)
pprint(resource)

{'data':                Statistic  Year  ...           Region of Origin    value
0        Tonne-kilometre  2016  ...  All regions and countries  11564.0
1        Tonne-kilometre  2016  ...                     Border    887.0
2        Tonne-kilometre  2016  ...                       West    839.0
3        Tonne-kilometre  2016  ...                   Mid-West   1088.0
4        Tonne-kilometre  2016  ...                 South-East   1077.0
...                  ...   ...  ...                        ...      ...
1810  Vehicle Kilometres  2020  ...                     Dublin     10.0
1811  Vehicle Kilometres  2020  ...                   Mid-East      4.0
1812  Vehicle Kilometres  2020  ...                    Midland      1.0
1813  Vehicle Kilometres  2020  ...           Northern Ireland      2.0
1814  Vehicle Kilometres  2020  ...            Other countries     55.0

[1815 rows x 5 columns],
 'format': 'pandas',
 'hashing': 'md5',
 'name': 'memory',
 'profile': 'tabular-data-resource',
 'sch

**Step 2: Extract**

Reads and extracts the data from the dataset

In [7]:
from frictionless import extract 
rows = extract(df)
pprint(rows)

Streaming output truncated to the last 5000 lines.
 {'Region of Destination': 'Midland',
  'Region of Origin': 'Border',
  'Statistic': 'Tonnes Carried',
  'Year': '2017',
  'value': Decimal('468.0')},
 {'Region of Destination': 'Midland',
  'Region of Origin': 'West',
  'Statistic': 'Tonnes Carried',
  'Year': '2017',
  'value': Decimal('430.0')},
 {'Region of Destination': 'Midland',
  'Region of Origin': 'Mid-West',
  'Statistic': 'Tonnes Carried',
  'Year': '2017',
  'value': Decimal('381.0')},
 {'Region of Destination': 'Midland',
  'Region of Origin': 'South-East',
  'Statistic': 'Tonnes Carried',
  'Year': '2017',
  'value': Decimal('542.0')},
 {'Region of Destination': 'Midland',
  'Region of Origin': 'South-West',
  'Statistic': 'Tonnes Carried',
  'Year': '2017',
  'value': Decimal('202.0')},
 {'Region of Destination': 'Midland',
  'Region of Origin': 'Dublin',
  'Statistic': 'Tonnes Carried',
  'Year': '2017',
  'value': Decimal('1368.0')},
 {'Region of Destination': 'Midlan

**Step 3: Validate**

Detects errors in the dataset and reports

In [8]:
from frictionless import validate 

report = validate(df)
pprint(report.task.scope)


['hash-count-error',
 'byte-count-error',
 'field-count-error',
 'row-count-error',
 'blank-header',
 'extra-label',
 'missing-label',
 'blank-label',
 'duplicate-label',
 'incorrect-label',
 'blank-row',
 'primary-key-error',
 'foreign-key-error',
 'extra-cell',
 'missing-cell',
 'type-error',
 'constraint-error',
 'unique-error']


In [9]:
from frictionless import Resource, FrictionlessException

try:
    resource = Resource(df)
except FrictionlessException as exception:
    pprint(exception.error)
    # Prints the SchemaError metadata in this case

In [10]:
from pprint import pprint
from frictionless import validate, checks

checks = [checks.sequential_value(field_name='Statistic')]
report = validate(df, checks=checks)
pprint(report.flatten(["rowPosition", "fieldPosition", "code", "note"]))

[[2, 1, 'sequential-value', 'the value is not sequential']]


**Step 4: Transform**

Various transformations can be applied to the dataset to modify the contents of the dataset or the metadata

In [11]:
from frictionless import Package, Resource, transform, steps

# Define source resource
source = Resource(df)
# Apply transform steps
target = transform(
    source,
    steps=[
        steps.table_normalize()
        
    ],
)

# Print resulting schema and data
pprint(target.schema)
print(target)
print(target.data)
pprint(source.read_lists())
pprint(target.read_lists())


Streaming output truncated to the last 5000 lines.
 ['Tonne-kilometre', '2020', 'Midland', 'Mid-East', 92.0],
 ['Tonne-kilometre', '2020', 'Midland', 'Midland', 144.0],
 ['Tonne-kilometre', '2020', 'Midland', 'Northern Ireland', 8.0],
 ['Tonne-kilometre', '2020', 'Midland', 'Other countries', 11.0],
 ['Tonne-kilometre',
  '2020',
  'Northern Ireland',
  'All regions and countries',
  395.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'Border', 59.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'West', 11.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'Mid-West', 11.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'South-East', 46.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'South-West', 7.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'Dublin', 131.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'Mid-East', 52.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'Midland', 26.0],
 ['Tonne-kilometre', '2020', 'Northern Ireland', 'Northern Ireland

In [12]:
def clean(resource):
  current = resource.to_copy

source = describe(df)
target = transform(
    source,
    steps=[
        clean,
        steps.table_write(path="TFA22.xlsx"),
    ],
)

In [13]:
#from pprint import pprint
#from frictionless import Package, Resource, transform, steps

#source = Resource(path=df)
#target = transform(
 #   source,
  #  steps=[
   #     steps.table_normalize(),
#    #    steps.table_pivot(f1="Statistic", f2="Year", aggfun=sum),
 #   ]
#)
#pprint(target.schema)
#pprint(target.read_rows())
#pprint(target)

In [14]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(df)
target = transform(
    source,
    steps=[
        steps.table_normalize(),
        steps.table_transpose(),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

{'fields': [{'name': 'Statistic', 'type': 'string'},
            {'name': 'Tonne-kilometre', 'type': 'any'},
            {'name': 'Tonne-kilometre2', 'type': 'any'},
            {'name': 'Tonne-kilometre3', 'type': 'any'},
            {'name': 'Tonne-kilometre4', 'type': 'any'},
            {'name': 'Tonne-kilometre5', 'type': 'any'},
            {'name': 'Tonne-kilometre6', 'type': 'any'},
            {'name': 'Tonne-kilometre7', 'type': 'any'},
            {'name': 'Tonne-kilometre8', 'type': 'any'},
            {'name': 'Tonne-kilometre9', 'type': 'any'},
            {'name': 'Tonne-kilometre10', 'type': 'any'},
            {'name': 'Tonne-kilometre11', 'type': 'any'},
            {'name': 'Tonne-kilometre12', 'type': 'any'},
            {'name': 'Tonne-kilometre13', 'type': 'any'},
            {'name': 'Tonne-kilometre14', 'type': 'any'},
            {'name': 'Tonne-kilometre15', 'type': 'any'},
            {'name': 'Tonne-kilometre16', 'type': 'any'},
            {'name': 'Tonne-k

FrictionlessException: ignored

In [15]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(df)
target = transform(
    source,
   steps=[
        steps.cell_set(field_name="Statistic", value="bad"),
        steps.table_validate(),
    ]
)
pprint(target.schema)
try:
  pprint(target.read_rows())
except Exception as exception:
  pprint(exception)

{'fields': [{'name': 'Statistic', 'type': 'string'},
            {'name': 'Year', 'type': 'string'},
            {'name': 'Region of Destination', 'type': 'string'},
            {'name': 'Region of Origin', 'type': 'string'},
            {'name': 'value', 'type': 'number'}]}
FrictionlessException('[source-error] The data source has not supported or has inconsistent contents: unsupported inline data')


In [16]:
source = Resource(df)
target = transform(
    source,
    steps=[
      steps.table_debug(function=print),
    ],
)
pprint(target)

{'data': <frictionless.transform.resource.DataWithErrorHandling object at 0x7f7681c3d490>,
 'format': 'inline',
 'name': 'memory',
 'profile': 'tabular-data-resource',
 'schema': {'fields': [{'name': 'Statistic', 'type': 'string'},
                       {'name': 'Year', 'type': 'string'},
                       {'name': 'Region of Destination', 'type': 'string'},
                       {'name': 'Region of Origin', 'type': 'string'},
                       {'name': 'value', 'type': 'number'}]},
 'scheme': ''}


In [17]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.field_move(name="Statistic", position=2),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

FrictionlessException: ignored

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(df)
target = transform(
    source,
    steps=[
        steps.resource_add(name='extra', path=df),
    ],
)

pprint(target.get('extra').schema)
pprint(target.get('extra').read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Package(resources=[Resource(name='main', path=df)])
target = transform(
    source,
    steps=[
        steps.resource_add(name='extra', path=df),
        steps.resource_transform(name='main', steps=[
            steps.table_merge(resource='extra'),
            steps.row_sort(field_names=['Statistic'])
        ]),
        steps.resource_remove(name="extra"),
    ],
)
pprint(target.resource_names)
pprint(target.get_resource('main').schema)
pprint(target.get_resource('main').read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.field_split(name="Region of Origin", to_names=["West", "East"], pattern="a"),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

In [ ]:
from pprint import pprint
from frictionless import Package, Resource, transform, steps

source = Resource(path=df)
target = transform(
    source,
    steps=[
        steps.field_filter(names=["Year", "Region of Destination"]),
    ]
)
pprint(target.schema)
pprint(target.read_rows())

**Excel:**

Different methods can be used to dump the dataset into an excel file for future use

In [18]:
from openpyxl import load_workbook
wb = load_workbook('TFA22.xlsx')
print(wb)

In [19]:
df.to_excel('TFA22.xlsx')
open('TFA22.xlsx')

<_io.TextIOWrapper name='TFA22.xlsx' mode='r' encoding='UTF-8'>

In [20]:
from frictionless import Parser

class HtmlParser(Parser):
    requires_loader = True
    supported_types = [
        "string",
    ]

    # Read

    def read_list_stream_create(self):
        pq = helpers.import_from_plugin("pyquery", plugin="html").PyQuery
        dialect = self.resource.dialect

        # Get Page content
        page = pq(self.loader.text_stream.read(), parser="html")

        # Find required table
        if dialect.selector:
            table = pq(page.find(dialect.selector)[0])
        else:
            table = page

        # Stream headers
        data = (
            table.children("thead").children("tr")
            + table.children("thead")
            + table.children("tr")
            + table.children("tbody").children("tr")
        )
        data = [pq(r) for r in data if len(r) > 0]
        first_row = data.pop(0)
        headers = [pq(th).text() for th in first_row.find("th,td")]
        yield headers

        # Stream data
        data = [pq(tr).find("td") for tr in data]
        data = [[pq(td).text() for td in tr] for tr in data if len(tr) > 0]
        yield from data

    # Write

    def write_row_stream(self, resource):
        source = resource
        target = self.resource
        html = "<html><body><table>\n"
        with source:
            for row in source.row_stream:
                if row.row_number == 1:
                    html += "<tr>"
                    for name in row.field_names:
                        html += f"<td>{name}</td>"
                    html += "</tr>\n"
                cells = row.to_list(types=self.supported_types)
                html += "<tr>"
                for cell in cells:
                    html += f"<td>{cell}</td>"
                html += "</tr>\n"
        html += "</table></body></html>"
        with tempfile.NamedTemporaryFile("wt", delete=False) as file:
            file.write(html)
        loader = system.create_loader(target)
        result = loader.write_byte_stream(file.name)
        return result
        pprint(target)

In [21]:
from frictionless import Parser

class HtmlParser(Parser):
    requires_loader = True
    supported_types = [
        "string",
    ]

    # Read

    def read_list_stream_create(self):
        pq = helpers.import_from_plugin("pyquery", plugin="html").PyQuery
        dialect = self.resource.dialect

        # Get Page content
        page = pq(self.loader.text_stream.read(), parser="html")

        # Find required table
        if dialect.selector:
            table = pq(page.find(dialect.selector)[0])
        else:
            table = page

        # Stream headers
        data = (
            table.children("thead").children("tr")
            + table.children("thead")
            + table.children("tr")
            + table.children("tbody").children("tr")
        )
        data = [pq(r) for r in data if len(r) > 0]
        first_row = data.pop(0)
        headers = [pq(th).text() for th in first_row.find("th,td")]
        yield headers

        # Stream data
        data = [pq(tr).find("td") for tr in data]
        data = [[pq(td).text() for td in tr] for tr in data if len(tr) > 0]
        yield from data

    # Write

    def write_row_stream(self, resource):
        source = resource
        target = self.resource
        html = "<html><body><table>\n"
        with source:
            for row in source.row_stream:
                if row.row_number == 1:
                    html += "<tr>"
                    for name in row.field_names:
                        html += f"<td>{name}</td>"
                    html += "</tr>\n"
                cells = row.to_list(types=self.supported_types)
                html += "<tr>"
                for cell in cells:
                    html += f"<td>{cell}</td>"
                html += "</tr>\n"
        html += "</table></body></html>"
        with tempfile.NamedTemporaryFile("wt", delete=False) as file:
            file.write(html)
        loader = system.create_loader(target)
        result = loader.write_byte_stream(file.name)
        return result
        pprint(target)

In [22]:
import os.path
from frictionless import Resource
resource = Resource(df)
from frictionless.plugins.excel import ExcelDialect
resource.write("File.xlsx", dialect=ExcelDialect(sheet='My Table'))

{'dialect': {'sheet': 'My Table'}, 'path': 'File.xlsx'}